<a href="https://colab.research.google.com/github/DhruvaBansal00/ConfidentMT/blob/master/NoisyChannel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install fairseq numpy

     |████████████████████████████████| 307kB 43.9MB/s 
     |████████████████████████████████| 61kB 10.2MB/s 
     |████████████████████████████████| 17.1MB 192kB/s 
  Created wheel for fairseq: filename=fairseq-0.9.0-cp36-cp36m-linux_x86_64.whl size=2033614 sha256=035464a39d929356e1ab9aad099d6cde4b59378342f3ffc1e5435ef6e8a01688
  Stored in directory: /root/.cache/pip/wheels/37/3e/1b/0fa30695dcba41e4b0088067fa40f3328d1e8ee78c22cd4766
Successfully built fairseq


In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!ls
%cd drive/My Drive/ConfidentMachineTranslation/flores/noisychannel
!ls
!pwd

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
drive  sample_data
/content/drive/My Drive/ConfidentMachineTranslation/flores/noisychannel
__init__.py  rerank_generate.py  rerank_score_bw.py  rerank_utils.py
__pycache__  rerank_options.py	 rerank_score_lm.py  subword-nmt
rerank_data  rerank.py		 rerank_tune.py
/content/drive/My Drive/ConfidentMachineTranslation/flores/noisychannel


Noisy Channel Decoding runner code


In [0]:
import glob
path = '../Ensembles/*'
ensemble_models = ""
for file in glob.glob(path):
  ensemble_models += file + ":"
ensemble_models = ensemble_models[:-1]

beam=20 #original: 50
num_trials=100 #original: 1000. Reduced due to computational constraints.
fw_name="ne-en"
bw_name="en-ne"
lm_name="en_lm"
data_dir="../data-bin/wiki_ne_en_bpe5000"
data_dir_name="ncd_without_ensembling"
lm="../language_models/lm/checkpoint_best.pt"
lm_bpe_code="../language_models/lm/bpe32k.code"
lm_dict="../language_models/lm/dict.txt"
batch_size=32
bw="../backward_models/April_17.pt"
# fw="../Ensembles/April_04.pt"
fw = ensemble_models
print(fw)
source_lang = "ne"
target_lang = "en"

../Ensembles/April_04.pt:../Ensembles/April_10.pt:../Ensembles/April_11.pt:../Ensembles/April_13.pt:../Ensembles/April_14.pt


In [0]:
# reranking with P(T|S) P(S|T) and P(T)
!python rerank_tune.py $data_dir --tune-param lenpen weight1 weight3  \
    --lower-bound 0 0 0 --upper-bound 3 3 3 --data-dir-name $data_dir_name \
    --num-trials $num_trials  --source-lang $source_lang --target-lang $target_lang --gen-model $fw \
    -n $beam --batch-size $batch_size --score-model2 $fw --score-model1 $bw \
    --backwards1 --weight2 1 \
    -lm $lm  --lm-dict $lm_dict  --lm-name en_newscrawl --lm-bpe-code $lm_bpe_code \
    --model2-name $fw_name --model1-name $bw_name --gen-model-name $fw_name --sacrebleu

STEP 1: generate predictions using the p(T|S) model with bpe
../data-bin/wiki_ne_en_bpe5000
Namespace(beam=20, bpe=None, cpu=False, criterion='cross_entropy', data='../data-bin/wiki_ne_en_bpe5000', dataset_impl=None, decoding_format=None, diverse_beam_groups=-1, diverse_beam_strength=0.5, empty_cache_freq=0, force_anneal=None, fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, gen_subset='valid', iter_decode_eos_penalty=0.0, iter_decode_force_max_iter=False, iter_decode_max_iter=10, lazy_load=False, left_pad_source='True', left_pad_target='False', lenpen=1, load_alignments=False, log_format=None, log_interval=1000, lr_scheduler='fixed', lr_shrink=0.1, match_source_len=False, max_len_a=0, max_len_b=200, max_sentences=32, max_source_positions=1024, max_target_positions=1024, max_tokens=4000, memory_efficient_fp16=False, min_len=1, min_loss_scale=0.0001, model_overrides='{}', momentum=0.99, nbest=20, no_beamable_mm=False, no_early_stop=False, no_progress_ba

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# reranking with P(T|S) and P(T). We haven't tried this yet - Dhruva (4/20).
!python rerank_tune.py $data_dir  --tune-param lenpen weight3 \
    --lower-bound 0 0 --upper-bound 3 3  --data-dir-name $data_dir_name  \
    --num-trials $num_trials  --source-lang $source_lang --target-lang $target_lang --gen-model $fw \
    -n $beam --batch-size $batch_size --score-model1 $fw \
    -lm $lm  --lm-dict $lm_dict  --lm-name en_newscrawl --lm-bpe-code $lm_bpe_code \
    --model1-name $fw_name --gen-model-name $fw_name --sacrebleu

In [0]:
# # to run with a preconfigured set of hyperparameters for the lenpen and model weights, using rerank.py instead.
!python rerank.py $data_dir \
    --lenpen 1.2 --weight1 1 --weight2 0 --weight3 0  \
    --data-dir-name $data_dir_name  --source-lang $source_lang --target-lang $target_lang --gen-model $fw \
    -n $beam --batch-size $batch_size --score-model2 $fw --score-model1 $bw --backwards1  \
    -lm $lm  --lm-dict $lm_dict  --lm-name en_newscrawl --lm-bpe-code $lm_bpe_code \
    --model2-name $fw_name --model1-name $bw_name --gen-model-name $fw_name --sacrebleu

Streaming output truncated to the last 5000 lines.
 Hypothesis = The popular Nashhta contains Golpa (the same is also called in other places).

Target = Prepared Exclusively in tribal regions, this cuisine uses oil and spices infrequently, except for pickle production and special occasions.
 Hypothesis = In Indigenous areas, this business uses an informally, except oil and massala, new products and special oil.

Target = Rajasthan is also influenced by the rajputs who were predominantly non vegetarians.
 Hypothesis = The kingdom is also affected from the capital that was ruler over the people.

Target = Seafood is common in the coastal region of the state.
 Hypothesis = The communication in the state's tentative area.

Target = Seafood plays a major role in the cuisine of the andaman and Nicobar Islands.
 Hypothesis = The Sea food has been released to the chief role in Armen and Nicobar islands.

Target = Some common ingredients used are mango powder, Tamarind, kokum flowers, and dried